<a href="https://colab.research.google.com/github/jonathan093084/analisis_datos/blob/main/Clase_Julio_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Revision y Limpieza de Datos

In [35]:
# Abrir el archivo e importar la data
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
df=pd.read_csv("diversified_job_postings_version2.csv")

In [36]:
#Se realiza la estadistica descriptiva de los datos
print(df.describe())

          salary_usd  remote_ratio  years_experience  job_description_length  \
count   15000.000000  15000.000000      15000.000000            15000.000000   
mean    82622.067467     55.256667          6.253200             1503.314733   
std     69276.725683     47.081651          5.545768              576.127083   
min     16869.000000      0.000000          0.000000              500.000000   
25%     35768.250000      0.000000          2.000000             1003.750000   
50%     56243.000000    100.000000          5.000000             1512.000000   
75%    101240.250000    100.000000         10.000000             2000.000000   
max    517110.000000    100.000000         19.000000             2499.000000   

       benefits_score  
count    15000.000000  
mean         6.468800  
std          1.761354  
min          1.000000  
25%          5.300000  
50%          6.500000  
75%          7.700000  
max         10.000000  


In [37]:
#Mostramos un resumen de la información del DataFrame
# Esto incluye el número de entradas, el tipo de datos de cada columna y la cantidad de valores no nulos     
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   job_id                  15000 non-null  object 
 1   job_title               15000 non-null  object 
 2   salary_usd              15000 non-null  int64  
 3   salary_currency         15000 non-null  object 
 4   experience_level        15000 non-null  object 
 5   employment_type         15000 non-null  object 
 6   company_location        15000 non-null  object 
 7   company_size            15000 non-null  object 
 8   employee_residence      15000 non-null  object 
 9   remote_ratio            15000 non-null  int64  
 10  required_skills         15000 non-null  object 
 11  education_required      15000 non-null  object 
 12  years_experience        15000 non-null  int64  
 13  industry                15000 non-null  object 
 14  posting_date            15000 non-null

In [38]:
#Revisamos si hay valores nulos en el DataFrame
print(df.isnull().sum())


job_id                    0
job_title                 0
salary_usd                0
salary_currency           0
experience_level          0
employment_type           0
company_location          0
company_size              0
employee_residence        0
remote_ratio              0
required_skills           0
education_required        0
years_experience          0
industry                  0
posting_date              0
application_deadline      0
job_description_length    0
benefits_score            0
company_name              0
dtype: int64


In [39]:
#Revisamos los tipos de datos de cada columna
# Esto es útil para asegurarnos de que los datos estén en el formato correcto para el análisis
print(df.dtypes)

job_id                     object
job_title                  object
salary_usd                  int64
salary_currency            object
experience_level           object
employment_type            object
company_location           object
company_size               object
employee_residence         object
remote_ratio                int64
required_skills            object
education_required         object
years_experience            int64
industry                   object
posting_date               object
application_deadline       object
job_description_length      int64
benefits_score            float64
company_name               object
dtype: object


In [40]:
#Revisamos los valores únicos de las columnas relevantes
# Esto nos ayuda a entender la diversidad de datos en cada columna y detectar posibles inconsistencias  
print(df[['job_title',
'salary_currency',
'experience_level',
'employment_type',
'company_location',
'company_size',
'employee_residence',
'remote_ratio',
'education_required',
'industry']].nunique())

job_title             20
salary_currency        3
experience_level       4
employment_type        4
company_location      20
company_size           3
employee_residence    20
remote_ratio           3
education_required     4
industry              15
dtype: int64


In [41]:
#Limpieza y transformación de datos

# 1. Eliminar columnas innecesarias
df= df.drop('job_description_length', axis=1, errors='ignore')

# 2. Convertir los valor4es de fecha a tipo datetime
df['posting_date'] = pd.to_datetime(df['posting_date'], errors='coerce')
df['application_deadline'] = pd.to_datetime(df['application_deadline'], errors='coerce')

# 3. Convertir la columna de salario a numérico
df['salary_usd'] = pd.to_numeric(df['salary_usd'], errors='coerce')

# 4. Calculamos la duración del proceso de aplicación en dias
df['application_duration_days'] = (df['application_deadline'] - df['posting_date']).dt.days

# 5. Separar habilidades como lista
df['required_skills'] = df['required_skills'].fillna('').apply(
    lambda x: [s.strip() for s in x.split(',') if s.strip()]
)

# 6. Renombrar niveles de experiencia
experience_map = {
    "SE": "Expert",
    "MI": "Intermediate",
    "EN": "Junior",
    "EX": "Director"
}
df['experience_level'] = df['experience_level'].astype(str).str.strip().replace(experience_map).astype('category')

# 7. Renombrar tipo de empleo
employment_map = {
    "PT": "Part-time",
    "FT": "Full-time",
    "CT": "Contract",
    "FL": "Freelance"
}
df['employment_type'] = df['employment_type'].astype(str).str.strip().replace(employment_map).astype('category')

# 8. Renombrar tamaño de empresa
size_map = {
    "S": "Small",
    "M": "Medium",
    "L": "Large"
}
df['company_size'] = df['company_size'].astype(str).str.strip().replace(size_map).astype('category')

# 9. Clasificación para remote_ratio
remote_map = {
    0: 'No remote',
    50: 'Hybrid',
    100: 'Fully remote'
}
df['remote_ratio'] = df['remote_ratio'].replace(remote_map).astype('category')

# 10. Categóricos iniciales
cat_cols = ['job_title', 'experience_level', 'employment_type', 'company_location',
            'company_size', 'employee_residence', 'remote_ratio', 'education_required', 'industry']
df[cat_cols] = df[cat_cols].astype('category')


print(df.head())

    job_id              job_title  salary_usd salary_currency  \
0  AI00001  AI Research Scientist       55355             USD   
1  AI00002   AI Software Engineer       41552             USD   
2  AI00003          AI Specialist       70594             USD   
3  AI00004           NLP Engineer       43020             USD   
4  AI00005          AI Consultant       90618             EUR   

  experience_level employment_type company_location company_size  \
0           Junior       Full-time          Germany        Large   
1           Junior       Full-time    United States        Small   
2     Intermediate       Full-time        Singapore        Large   
3           Junior       Freelance           Norway        Small   
4     Intermediate        Contract          Germany        Small   

  employee_residence  remote_ratio  \
0              China  Fully remote   
1            Ireland     No remote   
2        South Korea     No remote   
3              India  Fully remote   
4         

In [42]:

# --- Pregunta 1: Países con mejores salarios ---

# Agrupamos por ubicacion de compañia y calculamos el salario promedio
# Luego ordenamos de mayor a menor y seleccionamos los 15 primeros
top_salaries_by_country = (
    df.groupby('company_location')['salary_usd']
    .mean()
    .sort_values(ascending=False)
    .head(10)
    .reset_index()
)

fig = px.bar(
    top_salaries_by_country,
    x='company_location',
    y='salary_usd',
    title='Top 10 Ubicación de compañias con salarios promedio más altos de empleos IA',
    labels={'company_location': 'Compañias x País', 'salary_usd': 'Salario Promedio (USD)'},
    text='salary_usd',
    color='company_location',
    color_discrete_sequence=px.colors.sequential.Viridis
)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(xaxis_tickangle=-45, showlegend=False)

fig.show()

/var/folders/94/y35xtp8579d2q1w_frtz8cy40000gp/T/ipykernel_21984/4053551974.py:6: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [43]:
# --- Pregunta 2: Salario promedio según nivel de experiencia ---


# Agrupar salario promedio por nivel de experiencia
salary_by_experience = (
    df.groupby('experience_level')['salary_usd']
    .mean()
    .sort_values(ascending=False)
    .reset_index()
)

# Crear gráfico de barras con Plotly
fig = px.bar(
    salary_by_experience,
    x='experience_level',
    y='salary_usd',
    title='Salario promedio según nivel de experiencia',
    labels={'experience_level': 'Nivel de experiencia','salary_usd': 'Salario Promedio (USD)'},
    text='salary_usd',
    color='experience_level',
    color_discrete_sequence=px.colors.sequential.Plasma
)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(xaxis={'categoryorder': 'total descending'}, xaxis_tickangle=-45, showlegend=False)

# Mostrar gráfico
fig.show()

/var/folders/94/y35xtp8579d2q1w_frtz8cy40000gp/T/ipykernel_21984/3053100582.py:6: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [44]:
# --- Pregunta 3: Salario promedio por tamaño de empresa ---

# Agrupar salario promedio por tamaño de empresa
salary_by_company_size = (
    df.groupby('company_size')['salary_usd']
    .mean()
    .sort_values(ascending=False)
    .reset_index()
)

# Crear gráfico
fig_size = px.bar(
    salary_by_company_size,
    x='company_size',
    y='salary_usd',
    title='Salario promedio según tamaño de empresa',
    labels={'company_size': 'Tamaño de la empresa','salary_usd': 'Salario Promedio (USD)'},
    text='salary_usd',
    color='company_size',
    color_discrete_sequence=px.colors.sequential.Plasma
)
fig_size.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig_size.update_layout(xaxis={'categoryorder': 'total descending'},xaxis_tickangle=-45, showlegend=False)

# Mostrar gráfico
fig_size.show()


/var/folders/94/y35xtp8579d2q1w_frtz8cy40000gp/T/ipykernel_21984/3169863462.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [45]:
# --- Pregunta 4: Top 15 categorías de trabajo con mayores salarios ---

# Agrupar por título del trabajo y calcular salario promedio
top_roles_by_salary = (
    df.groupby('job_title')['salary_usd']
    .mean()
    .sort_values(ascending=False)
    .head(15)
    .reset_index()
)

# Gráfico con tonos cálidos
fig_roles = px.bar(
    top_roles_by_salary,
    x='salary_usd',
    y='job_title',
    orientation='h',
    title='Top 15 Trabajos con mayores salarios en IA',
    labels={'job_title': 'Categoría de trabajo', 'salary_usd': 'Salario Promedio (USD)'},
    text='salary_usd',
    color='salary_usd',  # Usamos el valor de salario como referencia de color
    color_continuous_scale=px.colors.sequential.Viridis
)
fig_roles.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig_roles.update_layout(yaxis={'categoryorder': 'total ascending'}, coloraxis_showscale=True)

fig_roles.show()


/var/folders/94/y35xtp8579d2q1w_frtz8cy40000gp/T/ipykernel_21984/254828643.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [46]:
# --- Pregunta 5: Salario promedio según tipo de empleo ---

# Agrupar por tipo de empleo y calcular salario promedio
salary_by_employment = (
    df.groupby('employment_type')['salary_usd']
    .mean()
    .sort_values(ascending=False)
    .reset_index()
)

# Gráfico de barras con escala Viridis
fig_employment = px.bar(
    salary_by_employment,
    x='employment_type',
    y='salary_usd',
    title='Salario promedio según tipo de empleo',
    labels={'employment_type': 'Tipo de empleo', 'salary_usd': 'Salario Promedio (USD)'},
    text='salary_usd',
    color='employment_type',
    color_discrete_sequence=px.colors.sequential.Plasma
)
fig_employment.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig_employment.update_layout(xaxis={'categoryorder': 'total descending'}, xaxis_tickangle=-45, showlegend=False)

fig_employment.show()


/var/folders/94/y35xtp8579d2q1w_frtz8cy40000gp/T/ipykernel_21984/3515477497.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [47]:
# --- Pregunta 6: Salario promedio según porcentaje de trabajo remoto ---

# Agrupar salario promedio por porcentaje de trabajo remoto (0, 50, 100)
salary_by_remote = (
    df.groupby('remote_ratio')['salary_usd']
    .mean()
    .sort_values(ascending=False)
    .reset_index()
)

# Gráfico de barras
fig_remote = px.bar(
    salary_by_remote,
    x='remote_ratio',
    y='salary_usd',
    title='Salario promedio según porcentaje de trabajo remoto',
    labels={'remote_ratio': 'Porcentaje de trabajo remoto (%)', 'salary_usd': 'Salario Promedio (USD)'},
    text='salary_usd',
    color='remote_ratio',
    color_discrete_sequence=px.colors.sequential.Plasma
)
fig_remote.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig_remote.update_layout(xaxis={'categoryorder': 'category descending'}, xaxis_tickangle=-45, showlegend=False)

fig_remote.show()


/var/folders/94/y35xtp8579d2q1w_frtz8cy40000gp/T/ipykernel_21984/3613055181.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [48]:
# --- Pregunta 7: Top 15 industrias con mayores salarios ---

# Agrupar por industria y calcular salario promedio (top 15)
salary_by_industry = (
    df.groupby('industry')['salary_usd']
    .mean()
    .sort_values(ascending=False)
    .head(15)
    .reset_index()
)

# Gráfico horizontal
fig_industry = px.bar(
    salary_by_industry,
    x='salary_usd',
    y='industry',
    orientation='h',
    title='Top 15 Sectores de la industria con mayores salarios en IA',
    labels={
        'industry': 'Sectores de la Industria',
        'salary_usd': 'Salario Promedio (USD)'
    },
    text='salary_usd',
    color='salary_usd',
    color_continuous_scale=px.colors.sequential.Viridis
)
fig_industry.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig_industry.update_layout(yaxis={'categoryorder': 'total ascending'},  coloraxis_showscale=True)

fig_industry.show()


/var/folders/94/y35xtp8579d2q1w_frtz8cy40000gp/T/ipykernel_21984/2367888231.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [49]:
# --- Pregunta 8: Top 15 países con mayor cantidad de ofertas laborales ---

# Agrupar por país de empresa y contar
offers_by_country = (
    df['company_location']
    .value_counts()
    .head(10)
    .reset_index()
)
offers_by_country.columns = ['company_location', 'Cantidad']  # asignación de nombres a las columnas

# Gráfico
fig_country = px.bar(
    offers_by_country,
    x='company_location',
    y='Cantidad',
    title='Top 10 Ubicación de compañias con mayor cantidad de ofertas laborales en IA',
    labels={'company_location': 'Compañias x País','Cantidad': 'Cantidad de ofertas'},
    text='Cantidad',
    color='company_location',
    color_discrete_sequence=px.colors.sequential.Viridis
)
fig_country.update_traces(texttemplate='%{text}', textposition='outside')
fig_country.update_layout(xaxis={'categoryorder': 'total descending'}, xaxis_tickangle=-45, showlegend=False)

fig_country.show()

In [ ]:
# --- Pregunta 9: Distribución de tipos de contrato por país ---

# Agrupación y conteo
contract_distribution = (
    df.groupby(['company_location', 'employment_type'])
    .size()
    .reset_index(name='Cantidad')
)


# Crear gráfico con orden aplicado
fig_contract = px.bar(
    contract_distribution,
    x='company_location',
    y='Cantidad',
    color='employment_type',
    title='Distribución de tipos de contrato por Ubicación de compañia',
    labels={'company_location': 'Compañias x País','num_offers': 'Cantidad de Ofertas', 'employment_type': 'Tipo de empleo'}, 
    #text='Cantidad',
    color_continuous_scale=px.colors.sequential.Plasma
)

fig_contract.update_layout(xaxis={'categoryorder': 'total descending'}, xaxis_tickangle=-45, showlegend=True)

fig_contract.show()

print(contract_distribution)


/var/folders/94/y35xtp8579d2q1w_frtz8cy40000gp/T/ipykernel_21984/1256476261.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



  company_location employment_type  Cantidad
0        Australia        Contract        82
1        Australia       Freelance       123
2        Australia       Full-time       399
3        Australia       Part-time        26
4          Austria        Contract        25
5          Austria       Freelance        22
6          Austria       Full-time        97
7          Austria       Part-time         7
8           Canada        Contract       192
9           Canada       Freelance       252


In [52]:
# Contar cantidad de empleados por país de residencia
residence_counts = (
    df['employee_residence']
    .value_counts()
    .reset_index()
)
residence_counts.columns = ['employee_residence', 'num_employees']

# Gráfico de barras
fig_residence = px.bar(
    residence_counts,
    x='employee_residence',
    y='num_employees',
    text='num_employees',
    title='Países donde residen los empleados de IA',
    labels={'employee_residence': 'País de residencia del empleado', 'num_employees': 'Cantidad de empleados'},
    color='employee_residence',
    color_discrete_sequence=px.colors.sequential.Viridis
)
fig_residence.update_traces(texttemplate='%{text}', textposition='outside')
fig_residence.update_layout(xaxis={'categoryorder': 'total descending'}, xaxis_tickangle=-45, showlegend=False)

fig_residence.show()

In [53]:
# Contar empresas por país
company_location_counts = (
    df['company_location']
    .value_counts()
    .reset_index()
)
company_location_counts.columns = ['company_location', 'num_companies']

# Gráfico de barras
fig_companies = px.bar(
    company_location_counts,
    x='company_location',
    y='num_companies',
    text='num_companies',
    title='Ubicacion de las compañias con empleos IA',
    labels={
        'company_location': 'Compañias x País',
        'num_companies': 'Cantidad de empresas'
    },
    color='company_location',
    color_discrete_sequence=px.colors.sequential.Viridis
)
fig_companies.update_traces(texttemplate='%{text}', textposition='outside')
fig_companies.update_layout(xaxis={'categoryorder': 'total descending'}, xaxis_tickangle=-45, showlegend=False)

fig_companies.show()

In [54]:
# --- Pregunta 10: Relación entre ubicación de la empresa y residencia del empleado ---

# Agrupar por país de empresa y país de residencia del empleado
location_relation = (
    df.groupby(['company_location', 'employee_residence'])
    .size()
    .reset_index(name='num_matches')
)

# Filtrar las 20 relaciones más frecuentes
top_location_matches = location_relation.sort_values('num_matches', ascending=False).head(20)

# Gráfico de burbujas
fig_location_relation = px.scatter(
    top_location_matches,
    x='company_location',
    y='employee_residence',
    title='Relación entre ubicación de la conpañia y residencia del empleado (Top 20)',
    labels={'company_location': 'Ubicación de la empresa','employee_residence': 'Residencia del empleado','num_matches': 'Coincidencias'},
    size='num_matches',
    color='num_matches',
    color_continuous_scale=px.colors.sequential.Viridis
)

fig_location_relation.update_layout(xaxis_tickangle=-45)
fig_location_relation.show()


/var/folders/94/y35xtp8579d2q1w_frtz8cy40000gp/T/ipykernel_21984/848900618.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [55]:
# --- Pregunta 10: Relación entre ubicación de la empresa y residencia del empleado ---

# Agrupar por país de empresa y país de residencia del empleado
location_relation = (
    df.groupby(['company_location', 'employee_residence'])
    .size()
    .reset_index(name='num_matches')
)

# Filtrar las 20 relaciones más frecuentes
top_location_matches = location_relation.sort_values('num_matches', ascending=False)

# Gráfico de burbujas
fig_location_relation = px.scatter(
    top_location_matches,
    x='company_location',
    y='employee_residence',
    title='Relación entre ubicación de la empresa y residencia del empleado',
    labels={
        'company_location': 'Ubicación de la empresa',
        'employee_residence': 'Residencia del empleado',
        'num_matches': 'Coincidencias'
    },
    size='num_matches',
    color='num_matches',
    color_continuous_scale=px.colors.sequential.Viridis
)

fig_location_relation.update_layout(xaxis_tickangle=-45)
fig_location_relation.show()


/var/folders/94/y35xtp8579d2q1w_frtz8cy40000gp/T/ipykernel_21984/2941318082.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [58]:
# --- Pregunta 11: Distribución del trabajo remoto o híbrido por país ---

# Agrupar por país y modalidad de trabajo
remote_distribution = (
    df.groupby(['company_location', 'remote_ratio'])
    .size()
    .reset_index(name='Cantidad')
)

# Gráfico de barras apiladas
fig_remote_distribution = px.bar(
    remote_distribution,
    x='company_location',
    y='Cantidad',
    color='remote_ratio',
    text='Cantidad',
    title='Frecuencia del trabajo remoto o híbrido por Ubicacion de la compañia',
    labels={
        'company_location': 'Compañias x País',
        'remote_ratio': 'Modalidad de trabajo',
        'num_offers': 'Cantidad de ofertas'
    },
    color_discrete_sequence=px.colors.sequential.Plasma
)

fig_remote_distribution.update_layout(xaxis={'categoryorder': 'total descending'}, xaxis_tickangle=-45, showlegend=True)
fig_remote_distribution.show()


/var/folders/94/y35xtp8579d2q1w_frtz8cy40000gp/T/ipykernel_21984/4285073463.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [59]:
# --- Pregunta 12: Top habilidades más demandadas por país ---

# Asegurarse de tener habilidades separadas
skills_by_country = df.explode('required_skills')

# Contar menciones globales por habilidad
top_skills_global = (
    skills_by_country['required_skills']
    .value_counts()
    .reset_index()
)
top_skills_global.columns = ['required_skills', 'count']

# Gráfico de barras horizontales
fig_top_skills = px.bar(
    top_skills_global,
    x='count',
    y='required_skills',
    orientation='h',
    title='Top habilidades más demandadas en IA (global)',
    labels={
        'count': 'Cantidad de menciones',
        'required_skills': 'Habilidad'
    },
    text='count',
    color='count',
    color_continuous_scale=px.colors.sequential.Viridis
)

fig_top_skills.update_traces(textposition='outside')
fig_top_skills.update_layout(yaxis={'categoryorder': 'total ascending'})
fig_top_skills.show()

In [60]:
# --- Pregunta 12: Top habilidades más demandadas por país ---

# Expandir habilidades
skills_by_country = df.explode('required_skills')

# Obtener las 10 habilidades más frecuentes
top_skills_pie = (
    skills_by_country['required_skills']
    .value_counts()
    .head(10)
    .reset_index()
)
top_skills_pie.columns = ['required_skills', 'count']

# Gráfico de torta
fig_pie_skills = px.pie(
    top_skills_pie,
    names='required_skills',
    values='count',
    title='Top 10 habilidades más demandadas en IA (global)',
    color_discrete_sequence=px.colors.sequential.Viridis
)

fig_pie_skills.update_traces(textposition='inside', textinfo='percent+label')
fig_pie_skills.show()

In [61]:
# --- Pregunta 12: Top habilidades más demandadas por país ---

# Expandir la lista de habilidades a una por fila
skills_by_country = df.explode('required_skills')

# Agrupar por país y habilidad
skill_demand = (
    skills_by_country.groupby(['company_location', 'required_skills'])
    .size()
    .reset_index(name='count')
)

# Seleccionar las combinaciones más frecuentes
top_skills_by_country = skill_demand.sort_values('count', ascending=False)

# Gráfico de burbujas
fig_skills = px.scatter(
    top_skills_by_country,
    x='company_location',
    y='required_skills',
    size='count',
    color='count',
    color_continuous_scale=px.colors.sequential.Viridis,
    title='Top habilidades más demandadas por país',
    labels={
        'company_location': 'País',
        'required_skills': 'Habilidad',
        'count': 'Demanda'
    }
)

fig_skills.update_layout(xaxis_tickangle=-45)
fig_skills.show()


/var/folders/94/y35xtp8579d2q1w_frtz8cy40000gp/T/ipykernel_21984/1124995495.py:8: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [62]:
# --- Pregunta 12: Top habilidades más demandadas por país ---
# Expandir habilidades por fila
skills_by_country = df.explode('required_skills')

# Seleccionar las 20 habilidades más frecuentes
top_skills = (
    skills_by_country['required_skills']
    .value_counts()
    .head(20)
    .index
)

# Filtrar y agrupar por país y habilidad
heatmap_data = (
    skills_by_country[skills_by_country['required_skills'].isin(top_skills)]
    .groupby(['company_location', 'required_skills'])
    .size()
    .reset_index(name='count')
)

# Crear mapa de calor con orden de país personalizado
fig_heatmap_sorted = px.density_heatmap(
    heatmap_data,
    x='company_location',
    y='required_skills',
    z='count',
    color_continuous_scale=px.colors.sequential.Viridis,
    title='Mapa de calor: demanda de habilidades por Ubicacion de compañia',
    labels={
        'company_location': 'Compañias x País',
        'required_skills': 'Habilidad',
        'count': 'Demanda'
    },

)

fig_heatmap_sorted.update_layout(xaxis={'categoryorder': 'total descending'},xaxis_tickangle=-45)
fig_heatmap_sorted.show()


/var/folders/94/y35xtp8579d2q1w_frtz8cy40000gp/T/ipykernel_21984/1569644739.py:16: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [63]:
# --- Pregunta 13: Evolución de la oferta de empleo en IA a lo largo del tiempo ---

# Agrupar por mes de publicación
job_posting_trend = (
    df.groupby(df['posting_date'].dt.to_period('M'))
    .size()
    .reset_index(name='num_postings')
)

# Convertir a fecha (inicio de cada mes)
job_posting_trend['posting_date'] = job_posting_trend['posting_date'].dt.to_timestamp()

# Gráfico de líneas
import plotly.express as px

fig_trend = px.line(
    job_posting_trend,
    x='posting_date',
    y='num_postings',
    title='Evolución de la oferta de empleo en IA a lo largo del tiempo',
    labels={
        'posting_date': 'Fecha',
        'num_postings': 'Número de ofertas publicadas'
    },
    markers=True
)

fig_trend.update_layout(yaxis_title='Ofertas publicadas', yaxis=dict(range=[0, 1000]) ,xaxis_title='Fecha')
fig_trend.show()


In [64]:
# --- Pregunta 14: Picos de publicación de vacantes por mes en 2024 ---
#Revisar esta pregunta debido a que no muestra información relevante debiodo a que la data no tiene suficientes registros de otros años

# Filtrar solo vacantes publicadas en 2024
df_2024 = df[df['posting_date'].dt.year == 2024].copy()

# Extraer nombre del mes
df_2024['posting_month'] = df_2024['posting_date'].dt.month_name()

# Contar publicaciones por mes y ordenar de mayor a menor
monthly_2024 = (
    df_2024['posting_month']
    .value_counts()
    .reset_index()
)
monthly_2024.columns = ['month', 'num_postings']

# Gráfico de barras
fig_peaks_2024 = px.bar(
    monthly_2024,
    x='month',
    y='num_postings',
    text='num_postings',
    title='Picos de publicación de vacantes por mes en 2024',
    labels={
        'month': 'Mes',
        'num_postings': 'Número de publicaciones'
    },
    color='num_postings',
    color_continuous_scale=px.colors.sequential.Viridis
)

fig_peaks_2024.update_traces(texttemplate='%{text}', textposition='outside')
fig_peaks_2024.update_layout(xaxis={'categoryorder': 'total descending'}, xaxis_tickangle=-45)
fig_peaks_2024.show()


In [65]:
# --- Pregunta 15: Evolución de las ofertas por modalidad de trabajo remoto a lo largo del tiempo ---

# Asegurar que 'posting_date' esté en formato datetime
df['posting_date'] = pd.to_datetime(df['posting_date'], errors='coerce')

# Agrupar por mes y modalidad de trabajo remoto
remote_trend = (
    df.groupby([df['posting_date'].dt.to_period('M'), 'remote_ratio'])
    .size()
    .reset_index(name='num_offers')
)

# Convertir a fecha real para el eje X
remote_trend['posting_date'] = remote_trend['posting_date'].dt.to_timestamp()

# Gráfico de líneas
fig_remote_trend = px.line(
    remote_trend,
    x='posting_date',
    y='num_offers',
    color='remote_ratio',
    markers=True,
    title='Evolución de las ofertas por modalidad de trabajo remoto a lo largo del tiempo',
    labels={
        'posting_date': 'Fecha',
        'num_offers': 'Número de ofertas',
        'remote_ratio': 'Modalidad de trabajo'
    },
    color_discrete_sequence=px.colors.sequential.Plasma
)

fig_remote_trend.update_layout(xaxis_title='Fecha', yaxis_title='Número de ofertas')
fig_remote_trend.show()


/var/folders/94/y35xtp8579d2q1w_frtz8cy40000gp/T/ipykernel_21984/858574634.py:8: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [66]:
# --- Pregunta 16: Evolución de la duración entre publicación y fecha límite de aplicación ---
# Agrupar por mes y calcular la duración promedio
duration_trend = (
    df.groupby(df['posting_date'].dt.to_period('M'))['application_duration_days']
    .mean()
    .reset_index()
)
duration_trend['posting_date'] = duration_trend['posting_date'].dt.to_timestamp()

# Gráfico de evolución
import plotly.express as px

fig_duration = px.line(
    duration_trend,
    x='posting_date',
    y='application_duration_days',
    title='Duración promedio entre publicación y fecha límite a lo largo del tiempo',
    labels={
        'posting_date': 'Fecha de publicación',
        'application_duration_days': 'Duración promedio (días)'
    },
    markers=True
)

fig_duration.update_layout(xaxis_title='Fecha', yaxis_title='Duración promedio (días)')
fig_duration.show()


In [67]:
# --- Pregunta 17: BeBeneficios promedio por sector de la compañía ---

# Agrupar por industria
benefits_by_industry = (df.groupby('industry')['benefits_score']
.mean()
.reset_index()
)
# Ordenar de mayor a menor
benefits_by_industry = benefits_by_industry.sort_values(by='benefits_score', ascending=False)

# Crear gráfica
fig_benefits = px.bar(
    benefits_by_industry,
    x='industry',
    y='benefits_score',
    title='Beneficio promedio por sector de la compañía',
    labels={'industry': 'Industria', 'benefits_score': 'Puntaje de beneficios'},
    text='benefits_score',
    color='industry',
    color_discrete_sequence=px.colors.sequential.Viridis
)

fig_benefits.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig_benefits.update_layout(xaxis={'categoryorder': 'total descending'}, xaxis_tickangle=-45, showlegend=False)
fig_benefits.show()

/var/folders/94/y35xtp8579d2q1w_frtz8cy40000gp/T/ipykernel_21984/3612138258.py:4: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [68]:
# --- Pregunta 18: Salario promedio por nivel de educación requerido ---
# Agrupar y ordenar
salary_by_education = df.groupby('education_required')['salary_usd'].mean().reset_index()
salary_by_education = salary_by_education.sort_values(by='salary_usd', ascending=False)

# Crear gráfica
fig = px.bar(
    salary_by_education,
    x='education_required',
    y='salary_usd',
    text='salary_usd',
    title='Salario promedio por nivel de educación',
    labels={
        'education_required': 'Nivel de Educación',
        'salary_usd': 'Salario Promedio (USD)'
    },
    color='education_required',
    color_discrete_sequence=px.colors.sequential.Viridis
)

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(xaxis_tickangle=-45, showlegend=False)

fig.show()

/var/folders/94/y35xtp8579d2q1w_frtz8cy40000gp/T/ipykernel_21984/3044986005.py:3: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [69]:
# --- Pregunta 18: Salario promedio por nivel de educación requerido ---

import plotly.graph_objects as go
# Agrupar
salary_stats = df.groupby('education_required')['salary_usd'].agg(['mean', 'median', 'count']).reset_index()
salary_stats = salary_stats.sort_values('mean', ascending=False)

# Tomar dos colores de la paleta Viridis
viridis_colors = px.colors.sequential.Viridis
color_media = viridis_colors[4]
color_mediana = viridis_colors[5]

# Crear gráfico con barras agrupadas
fig = go.Figure()

fig.add_trace(go.Bar(
    x=salary_stats['education_required'],
    y=salary_stats['mean'],
    name='Media',
    marker_color=color_media,
    text=[f'{x:.2f}' for x in salary_stats['mean']],
    textposition='outside',
))

fig.add_trace(go.Bar(
    x=salary_stats['education_required'],
    y=salary_stats['median'],
    name='Mediana',
    marker_color=color_mediana,
    text=[f'{x:.2f}' for x in salary_stats['median']],
    textposition='outside',

))

# Personalización
fig.update_layout(
    title='Salario promedio y mediano por nivel de educación',
    xaxis=dict(title='Nivel de Educación', tickangle=-45),
    yaxis=dict(title='Salario (USD)'),
    barmode='group',
    legend_title='Estadística',
    margin=dict(t=50, b=100),
    height=500
)

fig.show()

print(salary_stats.round(2))   

/var/folders/94/y35xtp8579d2q1w_frtz8cy40000gp/T/ipykernel_21984/2194254555.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



  education_required      mean   median  count
3                PhD  94040.86  63776.0   3678
2             Master  86408.15  59844.0   3748
1           Bachelor  79484.75  52751.0   3789
0          Associate  70917.64  48654.0   3785


In [70]:

#Realizar un análisis de la educación requerida por cargo

# Agrupar por cargo y educación
education_by_job = df.groupby(['job_title', 'education_required']).size().reset_index(name='count')

# Filtrar los 10 cargos más comunes
top_jobs = (df['job_title']
            .value_counts()
            .head(10).index)
filtered_data = education_by_job[education_by_job['job_title'].isin(top_jobs)]

# Gráfica de barras agrupadas
fig_education_required = px.bar(
    filtered_data,
    x='job_title',
    y='count',
    color='education_required',
    title='Nivel de educación requerido según el cargo',
    labels={'job_title': 'Cargo', 'count': 'Número de Ofertas', 'education_required': 'Nivel Educativo'},
    barmode='group',
    color_discrete_sequence=px.colors.sequential.Plasma,
    text='count'
)

fig_education_required.update_traces(texttemplate='%{text}', textposition='outside')
fig_education_required.update_layout(xaxis={'categoryorder': 'total descending'}, xaxis_tickangle=-45, showlegend=True)
fig_education_required.show()

/var/folders/94/y35xtp8579d2q1w_frtz8cy40000gp/T/ipykernel_21984/4063234380.py:4: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [ ]:
#Relacionar el beneficio con el sector de la empresa - ok
#Relacionar el nivel de educación con los salarios - ok
#Cargo relacionado con el nivel de educación requerido - ok
#Para el analisis de la duracion entre posteo y aplicacion:
#   relacionar con salarios
#   nivel de educación
#   con el cargo
#Relacionar salarios con años de experiencia, con nivel de educación
# Relacionar la habilidad con el salario
#Cambiar mapa de calor por tipo de industrias
#Realizar un grafico de bigotes para algun dato
#matriz de correlacion


#Maoa de calor de habilidades x pias cambiarlo a sector de empresa
#Años de experiencia x nivel de educación relaciionarla con el salario






In [71]:
# Relación entre años de experiencia (years_experience) y nivel de educación
import plotly.express as px
exp_edu_data = df.groupby(['years_experience', 'education_required']).size().reset_index(name='count')
fig_exp_edu = px.bar(
    exp_edu_data,
    x='years_experience',
    y='count',
    color='education_required',
    barmode='group',
    title='Relación entre años de experiencia y nivel de educación',
    labels={
        'years_experience': 'Años de experiencia',
        'count': 'Cantidad de ofertas',
        'education_required': 'Nivel de educación'
    },
    color_discrete_sequence=px.colors.sequential.Plasma,
    text='count'
    )
fig_exp_edu.update_traces(texttemplate='%{text}', textposition='outside')
fig_exp_edu.update_layout(xaxis_tickangle=-45, showlegend=True)
fig_exp_edu.show()

/var/folders/94/y35xtp8579d2q1w_frtz8cy40000gp/T/ipykernel_21984/103585860.py:3: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [73]:
# Gráfico de burbujas: años de experiencia vs nivel de educación
import plotly.express as px
bubble_data = df.groupby(['years_experience', 'education_required']).size().reset_index(name='count')
fig_bubble = px.scatter(
    bubble_data,
    x='years_experience',
    y='education_required',
    size='count',
    color='count',
    color_continuous_scale=px.colors.sequential.Viridis,
    title='Gráfico de burbujas: años de experiencia vs nivel de educación',
    labels={
        'years_experience': 'Años de experiencia',
        'education_required': 'Nivel de educación',
        'count': 'Cantidad de ofertas'
    }
)
fig_bubble.update_layout(xaxis_tickangle=-45)
fig_bubble.show()

/var/folders/94/y35xtp8579d2q1w_frtz8cy40000gp/T/ipykernel_21984/1873520457.py:3: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

